In [0]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Upload dataset
* sign up for a Kaggle account at https://www.kaggle.com.
* go to the 'Account' tab of your user profile https://www.kaggle.com/
* select 'Create API Token'.
* This will trigger the download of kaggle.json, a file containing your API credentials

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"phiztee","key":"71b8f9f82db0c8f264c1e3d3eed254b8"}'}

In [0]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 99% 1.14G/1.15G [00:15<00:00, 62.1MB/s]
100% 1.15G/1.15G [00:15<00:00, 80.3MB/s]


In [0]:
!apt install pv
!unzip -o /content/chest-xray-pneumonia.zip  | pv -l >/dev/null
os.remove('chest-xray-pneumonia.zip')
!unzip -o /content/chest_xray.zip  | pv -l >/dev/null
os.remove('chest_xray.zip')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Suggested packages:
  doc-base
The following NEW packages will be installed:
  pv
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 48.3 kB of archives.
After this operation, 123 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 pv amd64 1.6.6-1 [48.3 kB]
Fetched 48.3 kB in 0s (122 kB/s)
Selecting previously unselected package pv.
(Reading database ... 130812 files and directories currently installed.)
Preparing to unpack .../archives/pv_1.6.6-1_amd64.deb ...
Unpacking pv (1.6.6-1) ...
Setting up pv (1.6.6-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
2.00  0:00:10 [ 193m/s] [  <=>                                                 ]
11.8k 0:00:15 [ 762 /s] [               

In [0]:
# dimensions of our images.
img_width, img_height = 150, 150

In [0]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

## Define the model with Transfer learning of imagenet

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [0]:
model.layers

In [0]:
model.input

<tf.Tensor 'conv2d_7_input:0' shape=(?, 150, 150, 3) dtype=float32>

## Train Model

In [0]:
train_datagen=ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator=train_datagen.flow_from_directory('/content/chest_xray/train/',
                                                  target_size=(img_width, img_height),
                                                  color_mode='rgb',
                                                  batch_size=32,
                                                  class_mode='categorical', shuffle=True)

test_datagen=ImageDataGenerator(rescale=1. / 255)

test_generator=test_datagen.flow_from_directory('/content/chest_xray/test/',
                                                  target_size=(img_width, img_height),
                                                  color_mode='rgb',
                                                  batch_size=16,
                                                  class_mode='categorical', shuffle=True)

valid_datagen=ImageDataGenerator(rescale=1. / 255)

valid_generator=valid_datagen.flow_from_directory('/content/chest_xray/val/',
                                                  target_size=(img_width, img_height),
                                                  color_mode='rgb',
                                                  batch_size=16,
                                                  class_mode='categorical', shuffle=True)

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                   validation_data=test_generator,
                   validation_steps=test_generator.samples // test_generator.batch_size)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Epoch 1/5
163/163 [==============================] - 95s 582ms/step - loss: 0.2999 - acc: 0.8710 - val_loss: 0.4619 - val_acc: 0.7965
Epoch 2/5
163/163 [==============================] - 90s 553ms/step - loss: 0.2627 - acc: 0.8888 - val_loss: 0.3260 - val_acc: 0.8718
Epoch 3/5
163/163 [==============================] - 90s 551ms/step - loss: 0.2191 - acc: 0.9114 - val_loss: 0.3341 - val_acc: 0.8926
Epoch 4/5
163/163 [==============================] - 89s 548ms/step - loss: 0.2219 - acc: 0.9089 - val_loss: 0.4907 - val_acc: 0.8365
Epoch 5/5
163/163 [==============================] - 89s 545ms/step - loss: 0.1986 - acc: 0.9289 - val_loss: 0.4182 - val_acc: 0.8413


## Save the weight

In [0]:
model.save('chest-xray-pneumonia-created-model.h5')    
!zip -r created_model.zip 'chest-xray-pneumonia-created-model.h5'

updating: chest-xray-pneumonia-created-model.h5 (deflated 35%)


## Test the model

In [0]:
from keras.models import load_model
from keras.preprocessing import image

new_model = load_model("/content/chest-xray-pneumonia-created-model.h5")

In [0]:
def get_rez(pic):
  img = image.load_img(pic, target_size=(150,150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x /= 255.
  p_good,p_ill = np.around(new_model.predict(x), decimals=2)[0]
  return {'p_good':p_good,'p_ill':p_ill}

In [0]:
classes = train_generator.class_indices
print(classes)

{'NORMAL': 0, 'PNEUMONIA': 1}


In [0]:
ill_path = "/content/chest_xray/test/PNEUMONIA/" 
good_path = "/content/chest_xray/test/NORMAL/" 

In [0]:
ill_pic = ill_path + os.listdir(ill_path)[4]
good_pic = good_path + os.listdir(good_path)[4]

print(get_rez(ill_pic))
print(get_rez(good_pic))

{'p_good': 0.06, 'p_ill': 0.94}
{'p_good': 0.87, 'p_ill': 0.13}
